In [ ]:
!pip install facenet-pytorch

In [1]:
import pandas as pd
import shutil
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import sys
import os
import time
import torch.nn.functional as F
import os
import random
from torch.optim.lr_scheduler import ReduceLROnPlateau
from facenet_pytorch import InceptionResnetV1, fixed_image_standardization, training

In [2]:
data_transforms = transforms.Compose([
        np.float32,
        transforms.ToTensor(),
        fixed_image_standardization
])

image_dataset = datasets.ImageFolder('faces',  data_transforms)

In [3]:
os.listdir('faces')

['Neymar_Jr_football',
 'Sergio_Ramos_football',
 'L._Messi_football',
 'R._Lewandowski_football',
 'L._Suárez_football',
 'M._Neuer_football',
 'E._Hazard_football',
 'H._Kane_football',
 'T._Kroos_football']

In [ ]:
os.listdir('faces.')

In [4]:
if os.path.exists('faces/.DS_Store'):
    os.remove(os.path.join('faces', '.DS_Store' ))

if os.path.exists('faces/.ipynb_checkpoints'):
    os.remove(os.path.join('faces', '.ipynb_checkpoints' ))
    

In [5]:
img_inds = np.arange(len(image_dataset))

In [6]:
np.random.shuffle(img_inds)

In [7]:
train_inds = img_inds[:int(0.8 * len(img_inds))]
val_inds = img_inds[int(0.8 * len(img_inds)):]

In [8]:
from torch.utils.data import DataLoader, SubsetRandomSampler
from facenet_pytorch import InceptionResnetV1


In [9]:
train_loader = DataLoader(
    image_dataset,
    num_workers=2,
    batch_size=20,
    sampler=SubsetRandomSampler(train_inds),
    drop_last=True
)
val_loader = DataLoader(
    image_dataset,
    num_workers=2,
    batch_size=20,
    sampler=SubsetRandomSampler(val_inds),
    drop_last=True
)

In [10]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [11]:
loss_fn = torch.nn.CrossEntropyLoss()
metrics = {
    'acc': training.accuracy
}

In [12]:
 def complete_epoch(model, loss_fn, loader, optimizer=None, scheduler=None,  batch_metrics=metrics, device=device):   
    mode = 'Train' if model.training else 'Valid'
    print(mode)
    loss = 0
    metrics = {}
    for i_batch, (x, y) in enumerate(loader):
        x = x.to(device)
        y = y.to(device)
        y_pred = model(x)

        loss_batch = loss_fn(y_pred, y)
        

        if model.training:
            loss_batch.backward()
            optimizer.step()
            optimizer.zero_grad()


        
        metrics_batch = {}
        for metric_name, metric_fn in batch_metrics.items():
            metrics_batch[metric_name] = metric_fn(y_pred, y).detach().cpu()
            metrics[metric_name] = metrics.get(metric_name, 0) + metrics_batch[metric_name]
            

        loss_batch = loss_batch.detach().cpu()
        loss += loss_batch

    loss = loss / (i_batch + 1)
    metrics = {k: v / (i_batch + 1) for k, v in metrics.items()}

    return loss, metrics

In [13]:
len(image_dataset.class_to_idx)

9

In [14]:
image_dataset.class_to_idx

{'E._Hazard_football': 0,
 'H._Kane_football': 1,
 'L._Messi_football': 2,
 'L._Suárez_football': 3,
 'M._Neuer_football': 4,
 'Neymar_Jr_football': 5,
 'R._Lewandowski_football': 6,
 'Sergio_Ramos_football': 7,
 'T._Kroos_football': 8}

In [15]:
from facenet_pytorch import InceptionResnetV1

resnet = InceptionResnetV1(
    classify=True,
    num_classes=len(image_dataset.class_to_idx)
).to(device)

optimizer = optim.Adam(resnet.parameters(), lr=0.001)


In [17]:
import pickle
file_save = open("ids_to_player.pkl", "wb")
pickle.dump(image_dataset.class_to_idx, file_save)
file_save.close()


In [ ]:
dic = {value:key for key, value in output.items()}

In [ ]:
a_file = open("ids_to_player.pkl", "wb")
pickle.dump(dic, a_file)
a_file.close()


In [ ]:
for p in resnet.named_parameters():
    p[1].requires_grad = False


In [ ]:
resnet.logits.weight.requires_grad = True
resnet.logits.bias.requires_grad = True

In [ ]:
loss_fn = torch.nn.CrossEntropyLoss()
metrics = {
    'acc': training.accuracy
}


In [ ]:
best_loss = np.inf
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.5)
no_improvement = 0
for epoch in range(40):
    resnet.train()
    p_count = 0
    for p in resnet.named_parameters():
        if p_count >= len(list(resnet.named_parameters()))-2-3*epoch:
            p[1].requires_grad = True
    p_count += 1
    complete_epoch(resnet, loss_fn, train_loader, optimizer=optimizer, scheduler=None,  batch_metrics=metrics, device=device)
    resnet.eval()
    val_loss, acc =  complete_epoch(resnet, loss_fn, val_loader, optimizer=optimizer, scheduler=None,  batch_metrics=metrics, device=device)
    print(val_loss)
    print(best_loss)
    print(acc)
    if val_loss < best_loss:
        no_improvement = 0
        best_loss = val_loss
        print('hello')
        torch.save({
            'model_state_dict': resnet.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            }, '/content/gdrive/MyDrive/resnet_model_best_weights/model_best.pt'.format(str(epoch)))
    else:
        scheduler
        no_improvement += 1
        scheduler.step()
        if no_improvement == 5:
            print('No Improvement for 5 epochs, Early Stopping')
            break

Tracking Gradient For: logits.weight
Tracking Gradient For: logits.bias
Train
Valid
tensor(0.4383)
inf
{'acc': tensor(0.9211)}
hello
Tracking Gradient For: last_linear.weight
Tracking Gradient For: last_bn.weight
Tracking Gradient For: last_bn.bias
Tracking Gradient For: logits.weight
Tracking Gradient For: logits.bias
Train
Valid
tensor(0.2722)
tensor(0.4383)
{'acc': tensor(0.9211)}
hello
Tracking Gradient For: block8.branch1.2.bn.bias
Tracking Gradient For: block8.conv2d.weight
Tracking Gradient For: block8.conv2d.bias
Tracking Gradient For: last_linear.weight
Tracking Gradient For: last_bn.weight
Tracking Gradient For: last_bn.bias
Tracking Gradient For: logits.weight
Tracking Gradient For: logits.bias
Train
Valid
tensor(0.2764)
tensor(0.2722)
{'acc': tensor(0.9342)}
Tracking Gradient For: block8.branch1.1.bn.bias
Tracking Gradient For: block8.branch1.2.conv.weight
Tracking Gradient For: block8.branch1.2.bn.weight
Tracking Gradient For: block8.branch1.2.bn.bias
Tracking Gradient For